In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)   


In [50]:
main_url = 'https://sgulyano.github.io/eval/'

html_url = ['02226005.html',
            '08846113.html',
            '19689783.html',
            '24363682.html',
            '24674969.html',
            '26742703.html',
            '33472214.html',
            '33561138.html',
            '33565025.html',
            '41557226.html',
            '43951333.html',
            '44868136.html',
            '45435188.html',
            '48033334.html',
            '51435303.html',
            '53231654.html',
            '60189297.html',
            '61116032.html',
            '66887754.html',
            '68346670.html',
            '71728345.html',
            '79550445.html',
            '83255004.html',
            '94026030.html',
            '95860997.html',
            '96905621.html']
eval_url = [main_url + url for url in html_url]

def get_table_data(item):
    data = []
    table_body = item.find("tbody")
    rows = table_body.find_all("tr")
    for row in rows :
        cols = row.find_all(["td","th"])
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return data
    
dfs = []
for i in range(26):
    dfs.append(f"df{i}")


for j in range(26):
    page  = j
    r = requests.get(eval_url[page])
    soup = BeautifulSoup(r.content,"html.parser")
    tables = soup.find_all("table")
    head = pd.DataFrame(get_table_data(tables[0]))
    section = head.iloc[1,1]

# Adjust dataframe
    df = pd.DataFrame(get_table_data(tables[2]))
    df.dropna(how="all",inplace=True)
    df.columns = df.iloc[1,:]
    rows = [0,1]
    df.drop(rows,axis=0,inplace=True)
    df.drop(df[df.ที่.str.contains("รวม")].index , inplace= True)


    df["topic"] = ""
    ind = 0
    for i in df["ที่"]:
        if len(i) > 1:
            topic = i
            df.iloc[ind,16] = topic
            ind += 1
        else:
            df.iloc[ind,16] = topic
            ind +=1 
    df.dropna(inplace= True)


# change all column "ร้อยละ" name
    cols = []
    count = 5
    for column in df.columns:
        if column == 'ร้อยละ':
            cols.append(f'{count}(%)')
            count-=1
            continue
        cols.append(column)
    df.columns = cols

    for i in range(2,15):
        df.iloc[:,i] = pd.to_numeric(df.iloc[:,i])
    df.set_index("ที่" , inplace= True)
# add  detail to data frame

    df["degree"] = head.iloc[3,1]
    df["semester"] = head.iloc[2,1]  
    df["subject"] = head.iloc[0,1]
    df["section"] = head.iloc[1,1]
    df["นักศึกษาที่จดทะเบียน"] = head.iloc[4,1]
    df["นักศึกษาที่ประเมิน"] = head.iloc[5,1]
    df["url"] = eval_url[page]

    dfs[j] = df
    
# concat all websites
result = pd.concat([dfs[i] for i in range(26)])

# separate year & semester 
result["year"] = ""
result["sem"] = ""

count = 0
for i in result["semester"]:
    yr_splt = i.split("/")
    result["year"][count] = yr_splt[1]
    result["sem"][count] = yr_splt[0]
    count += 1

# display
result

,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,topic,degree,semester,subject,section,นักศึกษาที่จดทะเบียน,นักศึกษาที่ประเมิน,url,year,sem
ที่,,,,,,,,,,,,,,,,,,,,,
1,สอนอย่างเป็นระบบ,4,66.67,1,16.67,0,0.00,1,16.67,0,...,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1
2,สอนให้คิด วิเคราะห์ วิจารณ์,4,66.67,0,0.00,1,16.67,1,16.67,0,...,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1
3,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2,33.33,1,16.67,1,16.67,1,16.67,1,...,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1
4,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3,50.00,2,33.33,0,0.00,0,0.00,1,...,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1
5,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3,50.00,2,33.33,1,16.67,0,0.00,0,...,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,ปริญญาตรี โครงการพิเศษ,1/2562,FGL497,231863,43,6 (13.95%),https://sgulyano.github.io/eval/02226005.html,2562,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ผู้สอนแทรกเนื้อหาเกี่ยวกับคุณธรรมจริยธรรมในการ...,15,51.72,9,31.03,3,10.34,2,6.90,0,...,ส่วนที่ 2 ข้อคำถามของคณะ/หน่วยงาน,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),https://sgulyano.github.io/eval/96905621.html,2563,1
1,โสตทัศนูปกรณ์ช่วยสอนในห้องเรียน เช่น ไมโครโฟน ...,14,48.28,8,27.59,5,17.24,1,3.45,1,...,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),https://sgulyano.github.io/eval/96905621.html,2563,1
2,สภาพของห้องเรียนหรือห้องปฏิบัติการ เช่น แสง อุ...,13,44.83,11,37.93,4,13.79,0,0.00,1,...,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,ปริญญาตรี โครงการพิเศษ,1/2563,NYE553,231863,43,29 (67.44%),https://sgulyano.github.io/eval/96905621.html,2563,1


In [53]:
result.pivot_table(index=["degree", "year" , "sem" ,"subject","section" ])

0      0(%)  \
degree                 year sem subject section                       
ปริญญาตรี โครงการปกติ  2561 2   RMW014  556574   0.333333  1.110000   
                       2562 1   RMW014  556574   0.333333  0.353333   
                            2   RMW014  556574   8.750000  4.186250   
                       2563 2   RMW014  556574   1.250000  0.715000   
ปริญญาตรี โครงการพิเศษ 2561 2   NVI781  231863   0.000000  0.000000   
                                YRY790  231863   0.000000  0.000000   
                       2562 1   FGL497  231863   0.000000  0.000000   
                                YRY790  231863   0.000000  0.000000   
                            2   FGL497  231863   0.000000  0.000000   
                       2563 1   DRI970  231863   0.200000  0.714000   
                                        728025   0.000000  0.000000   
                                NYE553  231863   0.000000  0.000000   
                                YRY790  728025   0.100000  1.250000   
                            2   AZW892  231863   0.000000  0.000000   
                                FGL497  231863   0.000000  0.000000   
                                        696574   0.000000  0.000000   
                                        728025   0.000000  0.000000   
                                NVI781  231863   0.000000  0.000000   
                       2564 1   DRI970  231863   0.000000  0.000000   
                                NVI781  231863   0.000000  0.000000   
                                NYE553  231863   0.000000  0.000000   
                                XCM602  231863   0.000000  0.000000   
                                YRY790  231863   0.000000  0.000000   
                            2   AZW892  231863   0.000000  0.000000   
                                FGL497  231863   0.125000  0.312500   
ปริญญาโท โครงการพิเศษ  2562 2   DHT256  270981   0.000000  0.000000   

                                                         1      1(%)  \
degree                 year sem subject section                        
ปริญญาตรี โครงการปกติ  2561 2   RMW014  556574    0.833333  2.777500   
                       2562 1   RMW014  556574    1.666667  1.771667   
                            2   RMW014  556574   16.750000  8.013750   
                       2563 2   RMW014  556574    1.125000  0.642500   
ปริญญาตรี โครงการพิเศษ 2561 2   NVI781  231863    0.000000  0.000000   
                                YRY790  231863    0.000000  0.000000   
                       2562 1   FGL497  231863    0.150000  2.500500   
                                YRY790  231863    0.100000  0.833500   
                            2   FGL497  231863    0.000000  0.000000   
                       2563 1   DRI970  231863    0.150000  0.535500   
                                        728025    0.150000  0.625500   
                                NYE553  231863    0.400000  1.379500   
                                YRY790  728025    0.600000  7.500000   
                            2   AZW892  231863    0.125000  2.500000   
                                FGL497  231863    0.000000  0.000000   
                                        696574    0.000000  0.000000   
                                        728025    0.000000  0.000000   
                                NVI781  231863    0.000000  0.000000   
                       2564 1   DRI970  231863    0.500000  0.942500   
                                NVI781  231863    0.000000  0.000000   
                                NYE553  231863    0.000000  0.000000   
                                XCM602  231863    0.000000  0.000000   
                                YRY790  231863    0.250000  0.757500   
                            2   AZW892  231863    0.000000  0.000000   
                                FGL497  231863    0.125000  0.312500   
ปริญญาโท โครงการพิเศษ  2562 2   DHT256  270981    0.150000  2.500500   

                                                  

In [52]:
len(result["url"].unique())

26